## The script below creates a Supply Chain BOM based on Enginerring BOM using data from a out-dated supply chain BOM. It also enables the supply chain BOM to sync with the Engineering BOM. Developed by yifei.li@byton.com

In [ ]:
import pandas as pd
import numpy as np
import xlrd, openpyxl
import csv, datetime

## Load EBOM, Chi's Supply Chain BOM and Shaolong's BOM

In [ ]:
#Engineering BOM (EBOM). Updated weekly and has the most up-to-date Part Number and Revision. 
#Supply Chain BOM is based on EBOM with additional sourcing and supplier quality information.

EBOM = pd.read_excel("/home/yifei/Documents/BOM_tool/EBOM 8.23.18.xlsx")
# new_header = EBOM.iloc[0]
# EBOM = EBOM[1:]
# EBOM.columns = new_header
EBOM.columns = EBOM.columns.str.replace("\(R\)\ ", "") # trim (R) away from the header
print("Engineering VP BOM Shape: ",EBOM.shape)
EBOM.head(5)
# print(EBOM.columns.values)
# EBOM.shape





In [ ]:
print(EBOM.columns.values)

Removing Phantom part and Fasterners and filter only "Buy" part leaves us 1461 Unique PNs

In [ ]:
EBOM_filtered = EBOM[(EBOM["Purchased Part Type"]=="Buy") & (EBOM["Part Type"]!="Phantom") & (EBOM["System"]!="Y Fasteners")]
print(EBOM_filtered.shape)


In [ ]:
#Chi's SCBOM. We need the column names from Chi's SCBOM
SCBOM_chi = pd.read_excel("/home/yifei/Documents/BOM_tool/SCBOM.xlsx")
print("Supply Chain BOM Shape: ",SCBOM_chi.shape)
SCBOM_chi.head(20)


In [ ]:
SCBOM_chi.columns

In [ ]:
PreAP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="PAP"]
print("Pre AP BOM shape: ", PreAP_BOM_chi.shape)
AP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="AP"]
print("AP BOM shape: ", AP_BOM_chi.shape)
VP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="VP"]
print("VP BOM shape: ", VP_BOM_chi.shape)

#Active VP BOM with latest part revision
VP_BOM_Activate_chi = VP_BOM_chi[(VP_BOM_chi["PartActive"]=="Active")]
print("VP Active BOM: ", VP_BOM_Activate_chi.shape)

VPM_BOM_Activate_chi_removePhantomPart = VP_BOM_Activate_chi[VP_BOM_Activate_chi["Type"]!="Phantom"]
print("removed phatom", VPM_BOM_Activate_chi_removePhantomPart.shape)

                                                   


In [ ]:
#Load shaolong's SC BOM which has the most sourcing information updated by buyers and SQEs. 
SCBOM_shaolong = pd.read_excel("/home/yifei/Documents/BOM_tool/Shaolong BOM.xlsx", index_col=None)
# SCBOM_shaolong.reset_index()
SCBOM_shaolong

In [ ]:
SCBOM_shaolong.columns.values

In [ ]:
print("Supply Chain BOM Shape: ",SCBOM_shaolong.shape)

## Step 0: Create a blank Supply Chain BOM using EBOM

In [ ]:
# copy EBOM to the new SCBOM
print("EBOM shape is: ", EBOM.shape)
SCBOM = EBOM.copy()
print("SCBOM shape is: ", SCBOM.shape)
#specify SCBOM columns
SCBOM.columns.values

In [ ]:
ebom_column_header = EBOM.columns.values

sourcing_column_header = ["Supplier Name", "Supplier Contact Person", "Supplier Contact Email",
                          "Supplier Manufacturing Location", "GSM", "GSM Manager", "Engineer", 
                          "Engineer Manager", "RFQ Sent", "RFQ Received", "LOI Received", 
                          "Sourcing Council Scheduled Date", "SourcingCouncilAprovedDate", 
                          "DSA", "GTC", "Award Letter", "Purchae Requisition", "Purchase Order",
                          "Sourcing Status",]

logistics_column_header = ["First Pickup Date", "Last Pickup Date", "CCC Required", "Piece Cost", 
                           "Custom Compliance Completed", "Packaging Proposal Approved"]

quality_column_header = ["SQE", "Priority", "APQP Kick Off", "APQP Status","Tooling Lead Time", 
                     "Tooling Kick Off Date", "Tooling Completion Date", "Home Line", "FTN", 
                     "Grain", "AAR", "Run At Rate", "PV Completion Date", "Phase 1 PSW Date", 
                     "Phase 3 PSW Date", "Comments"]

log_column_header = ["Engineering Release Estimtae", "Engineering Release Actual", "Assembly Station", 
                     "Maturity", "Configuration", "Part Creation Date", "Last Modified Date", "Part Status", 
                     "Part Active"]

column_header = sourcing_column_header + logistics_column_header + quality_column_header + log_column_header

In [ ]:
# add additional columns
def add_columns(df, columns):
    df2 = pd.DataFrame(columns=columns)
    df = df.append(df2, sort=False, verify_integrity=True)
    return df

In [ ]:
SCBOM = add_columns(SCBOM, column_header)
SCBOM.shape

## Step 1: Update new SCBOM using Shaolong's BOM

In [ ]:
df = pd.DataFrame({'BoolCol': [True, False, False, True, True], 'Name': [1,2,3,4,5]},
       index=[10,20,30,40,50])
idx = df.index[df['BoolCol']]
idx

In [ ]:
df.loc[idx[0]]["Name"]

In [ ]:
df

In [ ]:
df.loc[(df["Name"]==2) & (df["BoolCol"]==False)].index.values

In [ ]:
df.loc[[10], ["Name"]] = df.loc[[20], 'Name'].values
df

In [ ]:
# save dataframe to Excel
def save(df):
    #https://stackoverflow.com/questions/28837057/pandas-writing-an-excel-file-containing-unicode-illegalcharactererror
    df = df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

    writer = pd.ExcelWriter('Supply Chain BOM.xlsx')
    df.to_excel(writer, sheet_name="Supply Chain BOM", na_rep="" )
    writer.save()

#     df.to_csv("Supply Chain BOM.csv",quoting=csv.QUOTE_NONE, escapechar="\\")

In [ ]:
# search PN and Rev in shaolong's SCBOM
def search(PN):
    # shaolong's data doesn't have Rev.
#     df = SCBOM_shaolong.loc[(SCBOM_shaolong["BYTON P/N"]==PN) & (SCBOM_shaolong["BYTON REVISION"]==Rev)]
    df = SCBOM_shaolong.loc[(SCBOM_shaolong["BYTON P/N"]==PN)]
    index = df.index
    if (len(index)==1):
        return index
    else:
        #return the first index if duplicate PN found
        return index[0:1]
        

In [ ]:
print(search("AA-AF003416NN-00"))

In [ ]:
print(search("AA-YF000148NN-00"))

In [ ]:
def replace_bracket(data):
    value = str(data).replace('[', '').replace(']', '').replace('\'', '')
    if value == 'nan':
        return ""
    return value

def change_datetime_to_string(data):
    if data.size == 0:
        return ""
    elif (data == np.array([' '])):
        return ""
    elif data == "nan":
        return ""
    else:
        return data[0]
    
#     if data == np.array([' ']):
#         return ""
#     if data == "nan":
#         return ""
#     else:
#         return data[0].strftime('%m/%d/%Y')

In [ ]:
# copy Shaolong's BOM info and paste into SCBOM
def copy_and_paste_row(df1, index1, df2, index2):
    #sourcing
    df1.loc[index1,["Engineer"]] = replace_bracket(df2.loc[index2,["ENGINEER"]].values)
    df1.loc[index1,["GSM"]] = replace_bracket(df2.loc[index2,["GSM"]].values)
    df1.loc[index1,["GSM Manager"]] = replace_bracket(df2.loc[index2,["Senior GSM"]].values)
    df1.loc[index1,["Supplier Name"]] = replace_bracket(df2.loc[index2,["VP\nSUPPLIER\n(Name)"]].values)
    df1.loc[index1,["Sourcing Status"]] = replace_bracket(df2.loc[index2,["Supplier Sourced?"]].values)
    df1.loc[index1,["Supplier Contact Person"]] = replace_bracket(df2.loc[index2,["Supplier \nContact Person"]].values)
    df1.loc[index1,["Supplier Contact Email"]] = replace_bracket(df2.loc[index2,["Supplier \nContact E-mail"]].values)
    df1.loc[index1,["Supplier Manufacturing Location"]] = replace_bracket(df2.loc[index2,["Supplier Location"]].values)
    df1.loc[index1,["GTC"]] = replace_bracket(df2.loc[index2,["GTC & Contract"]].values)

    
    #Logistics
    
    #SQE
    df1.loc[index1,["SQE"]] = replace_bracket(df2.loc[index2,["SQE"]].values)
    df1.loc[index1,["Priority"]] = replace_bracket(df2.loc[index2,["SQE Priority?"]].values)
    df1.loc[index1,["APQP Kick Off"]] = replace_bracket(df2.loc[index2,["APQP \nKICK OFF"]].values)
    df1.loc[index1,["APQP Status"]] = replace_bracket(df2.loc[index2,["Unnamed: 33"]].values)
    df1.loc[index1,["Tooling Lead Time"]] = replace_bracket(df2.loc[index2,["Tooling\nLead time"]].values)
    df1.loc[index1,["Tooling Kick Off Date"]] = change_datetime_to_string(df2.loc[index2,["Tool Kick off Plan"]].values)
    df1.loc[index1,["Tooling Completion Date"]] = change_datetime_to_string(df2.loc[index2,["Hard tool Completion"]].values)
    df1.loc[index1,["Home Line"]] = change_datetime_to_string(df2.loc[index2,["Home line"]].values)
    df1.loc[index1,["FTN"]] = replace_bracket(df2.loc[index2,["FTN"]].values)
    df1.loc[index1,["Grain"]] = replace_bracket(df2.loc[index2,["Grain"]].values)
    df1.loc[index1,["FTN"]] = replace_bracket(df2.loc[index2,["FTN"]].values)
    df1.loc[index1,["AAR"]] = replace_bracket(df2.loc[index2,["AAR"]].values)
    df1.loc[index1,["Run At Rate"]] = change_datetime_to_string(df2.loc[index2,["Run at rate"]].values)
    df1.loc[index1,["PV Completion Date"]] = change_datetime_to_string(df2.loc[index2,["PV Completion"]].values)
    df1.loc[index1,["Phase 1 PSW Date"]] = change_datetime_to_string(df2.loc[index2,["Phase 1 PSW"]].values)
    df1.loc[index1,["Phase 3 PSW Date"]] = change_datetime_to_string(df2.loc[index2,["Phase 3 PSW"]].values)
    df1.loc[index1,["Comments"]] = replace_bracket(df2.loc[index2,["Comments"]].values)
    
    
    #log
#   log_column_header = ["Engineering Release Estimtae", "Engineering Release Actual", "Assembly Station", 
#                      "Maturity", "Configuration", "Part Creation Date", "Last Modified Date", "Part Status", 
#                      "Part Active"]
    df1.loc[index1,["Part Status"]] = "Latest revision"
    df1.loc[index1,["Part Active"]] = "Active"
        
        
    return df1
    
    
    
     

In [ ]:
print(str(SCBOM_shaolong.loc[1000, ["GSM"]].values))

In [ ]:
print(str(SCBOM_shaolong.loc[1000, ["GSM"]].values).replace('[', '').replace(']', '').replace('\'', ''))

In [ ]:
SCBOM_shaolong.loc[:, ["Tool Kick off Plan"]].values

In [ ]:
SCBOM_shaolong.loc[41, ["Tool Kick off Plan"]].values[0]

In [ ]:
# update SCBOM with inputs from Shaolong's BOM
for index, row in SCBOM.iterrows():
    #loop through EBOM by PN
    PN = row["Title"]
    Rev = row["Revision"]
    #find out if this PN and Rev exist in Shaolong's BOM, and return the index
    index_shaolong = search(PN)
    #if not found
    if (index_shaolong.values.size == 0):
        #do nothing
        continue
    #if PN found, update SCBOM with Shaolong's BOM info
    else:
        #copy shaolong's BOM info to SCBOM
        copy_and_paste_row(SCBOM, index, SCBOM_shaolong, index_shaolong)


        
            
        
        
    
    

In [ ]:
SCBOM.loc[[21]]

In [ ]:
save(SCBOM)

## Create a new jupyter notebook for this step
### Step 2: Maintain and update SCBOM using updated EBOM

In [43]:
import pandas as pd
import numpy as np
import xlrd, openpyxl
import csv, datetime

In [44]:
#load EBOM
def load():
    EBOM = pd.read_excel("/home/yifei/Documents/BOM_tool/EBOM 8.23.18.xlsx")
    EBOM.columns = EBOM.columns.str.replace("\(R\)\ ", "") # trim (R) away from the header
#load Supply Chain BOM
    SCBOM = pd.read_excel("/home/yifei/Documents/BOM_tool/Supply Chain BOM.xlsx")
    return EBOM, SCBOM


In [45]:
# search PN and Rev in SCBOM
def search(df, PN, Rev):
    df = df.loc[(df["Title"]==PN) & (df["Revision"]==Rev)]
    index = df.index
    if (len(index)==1):
        return index
    else:
        #return the first index if duplicate PN found
        return index[0:1]
        

In [46]:
def replace_bracket(data):
    value = str(data).replace('[', '').replace(']', '').replace('\'', '')
    if value == 'nan':
        return ""
    return value

def change_datetime_to_string(data):
    if data.size == 0:
        return ""
    elif (data == np.array([' '])):
        return ""
    elif data == "nan":
        return ""
    else:
        return data[0]
    
#     if data == np.array([' ']):
#         return ""
#     if data == "nan":
#         return ""
#     else:
#         return data[0].strftime('%m/%d/%Y')

In [47]:
# copy Shaolong's BOM info and paste into SCBOM
def copy_and_paste_row(df1, index1, df2, index2):
        
    df1.loc[index1,["Identifier"]] = replace_bracket(df2.loc[index2,["Identifier"]].values)
    df1.loc[index1,["Title"]] = replace_bracket(df2.loc[index2,["Title"]].values)
    df1.loc[index1,["Revision"]] = replace_bracket(df2.loc[index2,["Revision"]].values)
    df1.loc[index1,["Description"]] = replace_bracket(df2.loc[index2,["Description"]].values)
    df1.loc[index1,["QTY"]] = replace_bracket(df2.loc[index2,["QTY"]].values)
    df1.loc[index1,["UOM"]] = replace_bracket(df2.loc[index2,["UOM"]].values)
    df1.loc[index1,["Purchased Part Type"]] = replace_bracket(df2.loc[index2,["Purchased Part Type"]].values)
    df1.loc[index1,["UOM"]] = replace_bracket(df2.loc[index2,["UOM"]].values)
    df1.loc[index1,["Maturity"]] = replace_bracket(df2.loc[index2,["Maturity"]].values)
    df1.loc[index1,["Part Type"]] = replace_bracket(df2.loc[index2,["Part Type"]].values)
    df1.loc[index1,["System"]] = replace_bracket(df2.loc[index2,["System"]].values)
    df1.loc[index1,["SubSystem"]] = replace_bracket(df2.loc[index2,["SubSystem"]].values)
    df1.loc[index1,["Legacy Part Number"]] = replace_bracket(df2.loc[index2,["Legacy Part Number"]].values)
    df1.loc[index1,["Legacy Part Revision"]] = replace_bracket(df2.loc[index2,["Legacy Part Revision"]].values)
    df1.loc[index1,["Configuration"]] = replace_bracket(df2.loc[index2,["Configuration"]].values)

    return df1

In [48]:
# save dataframe to Excel
def save(df):
    #https://stackoverflow.com/questions/28837057/pandas-writing-an-excel-file-containing-unicode-illegalcharactererror
    df = df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

    writer = pd.ExcelWriter('Updated Supply Chain BOM.xlsx')
    df.to_excel(writer, sheet_name="Updated Supply Chain BOM", na_rep="" )
    writer.save()

#     df.to_csv("Supply Chain BOM.csv",quoting=csv.QUOTE_NONE, escapechar="\\")

In [49]:
EBOM, SCBOM = load()
for index, row in SCBOM.iterrows():
    PN = row["Title"]
    Rev = row["Revision"]
    index_EBOM = search(EBOM, PN, Rev)

    if (index_EBOM.size == 0):
        # if not found, deactivate the part
        print("looped in")
        SCBOM.loc[index, ["Part Active"]] = "Inactivate"
        SCBOM.loc[index, ["Part Status"]] = "Removed"
        SCBOM.loc[index, ["Last Modified Date"]] = datetime.datetime.now()

    else:
        # if found, copy EBOM values to SCBOM and continue
        df = copy_and_paste_row(SCBOM, index, EBOM, index_EBOM)



    #check if EBOM PN quantities match SCBOM activate PN quantities
    #if matched, do nothing, continue

#     print("EBOM shape: ", EBOM.shape)
#     print("SCBOM shape: ", SCBOM.shape)
    #check if EBOM PN quantities match SCBOM activate PN quantities
#     if (EBOM.shape[0] == SCBOM.shape[0]):
#         print("SCBOM successfully updated")
#     else:
#         #if not matched, find out new PN added to EBOM and added them to SCBOM
#         print("failed")

save(SCBOM)


looped in
